In [32]:
import os

file_path = "Inst1"
for filename in os.listdir(file_path):
    print("\nSiguiente archivo\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 0
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            line = next(file)
            n_customers = line.strip()
            print("Número de customers: " + n_customers)
            line = next(file)
            n_available_depots = line.strip()
            print("Número de depots disponibles: " + n_available_depots)
            line = next(file)
            line = next(file)
            
            #coordenadas depots
            while(line.strip()):
                coordinates_depots.append(line.split())
                
                line = next(file)
            
            print("Coordenadas depots:")
            print(coordinates_depots)
            
            line = next(file)
            
            #Coordenadas customers
            while(line.strip()):
                coordinates_customers.append(line.split())
                
                line = next(file)
            
            print("Coordenadas customers:")
            print(coordinates_customers)
            
            line = next(file)
            
            #Capacidad de vehículos
            v_capacity = line.strip()
            print("Capacidad vehículos: " + v_capacity)
            line = next(file)
            line = next(file)
            
            #Capacidad de los depots
            while(line.strip()):
                capacities_depots.append(line.split())
                
                line = next(file)
            
            print("Capacidades depots:")
            print(capacities_depots)
            
            line = next(file)
            
            while(line.strip()):
                customers_demands.append(line.strip())
                
                line = next(file)
            
            print("Demandas customers:")
            print(customers_demands)
            
            line = next(file)
            
            while(line.strip()):
                opening_costs_depots.append(line.split())
                
                line = next(file)
            
            print("Costos depots:")
            print(opening_costs_depots)
            
            line = next(file)
            
            
            opening_cost_route = line.strip()
                
            line = next(file)
            
            print("opening cost route:")
            print(opening_cost_route)
            
            line = next(file)
            
            zero_or_not = line.strip()
            
            print("zero o uno: " + zero_or_not)


Siguiente archivo

Número de customers: 100
Número de depots disponibles: 10
Coordenadas depots:
[['2', '10'], ['18', '44'], ['49', '8'], ['24', '29'], ['18', '24'], ['14', '12'], ['4', '39'], ['32', '30'], ['7', '29'], ['40', '48']]
Coordenadas customers:
[['12', '38'], ['32', '17'], ['29', '12'], ['25', '20'], ['2', '30'], ['31', '46'], ['5', '13'], ['13', '36'], ['38', '6'], ['29', '11'], ['38', '47'], ['9', '47'], ['11', '18'], ['17', '31'], ['19', '6'], ['12', '7'], ['5', '14'], ['40', '36'], ['9', '45'], ['24', '43'], ['48', '45'], ['50', '13'], ['8', '43'], ['42', '23'], ['43', '14'], ['43', '6'], ['49', '4'], ['35', '41'], ['31', '42'], ['19', '24'], ['26', '29'], ['43', '47'], ['37', '36'], ['30', '3'], ['8', '34'], ['18', '40'], ['40', '10'], ['4', '27'], ['24', '26'], ['41', '17'], ['36', '22'], ['6', '6'], ['39', '40'], ['18', '29'], ['20', '3'], ['27', '3'], ['8', '26'], ['13', '37'], ['9', '6'], ['3', '28'], ['5', '50'], ['5', '28'], ['40', '5'], ['16', '28'], ['46', '41